In [308]:
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC

df=pd.read_csv("/resources/data/Interest Rate Prediction/train.csv")
df_test=pd.read_csv("/resources/data/Interest Rate Prediction/test.csv")

In [309]:
df.head()

,Loan_ID,Loan_Amount_Requested,Loan_Grade,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Area_Type,Gender,Interest_Rate
0,10000001,"7,000",A,< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,B,Female,1
1,10000002,"30,000",A,4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,A,Female,3
2,10000003,"24,725",D,7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,B,Male,3
3,10000004,"16,000",D,< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,A,Male,3
4,10000005,"17,000",A,8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,B,Female,1


In [310]:
df.describe()

/usr/local/lib/python3.5/dist-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Loan_ID,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Interest_Rate
count,1.643090e+05,1.392070e+05,164309.000000,164309.000000,75930.000000,164309.000000,164309.000000,164309.000000
mean,1.008216e+07,7.333116e+04,17.207189,0.781698,34.229356,11.193818,25.067665,2.158951
std,4.743207e+04,6.037750e+04,7.845083,1.034747,21.761180,4.991813,11.583067,0.738364
min,1.000000e+07,4.000000e+03,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000
25%,1.004108e+07,NaN,11.370000,0.000000,NaN,8.000000,17.000000,2.000000
50%,1.008216e+07,NaN,16.840000,0.000000,NaN,10.000000,23.000000,2.000000
75%,1.012323e+07,NaN,22.780000,1.000000,NaN,14.000000,32.000000,3.000000
max,1.016431e+07,7.500000e+06,39.990000,8.000000,180.000000,76.000000,156.000000,3.000000


In [356]:
df_test.describe()

,Loan_ID,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Number_Open_Accounts,Total_Accounts
count,1.095410e+05,109541.000000,1.095410e+05,109541.000000,109541.000000,109541.000000,109541.000000
mean,1.021908e+07,14358.475137,7.346161e+04,17.228969,0.788810,11.174337,25.068440
std,3.162191e+04,8299.811736,5.116734e+04,7.847310,1.039903,4.946314,11.599639
min,1.016431e+07,500.000000,3.000000e+03,0.000000,0.000000,0.000000,2.000000
25%,1.019170e+07,8000.000000,4.867100e+04,11.350000,0.000000,8.000000,17.000000
50%,1.021908e+07,12000.000000,7.000000e+04,16.860000,0.000000,10.000000,24.000000
75%,1.024646e+07,20000.000000,8.200000e+04,22.780000,1.000000,14.000000,32.000000
max,1.027385e+07,35000.000000,7.446395e+06,39.990000,8.000000,84.000000,124.000000


In [311]:
df.apply(lambda x: sum(x.isnull()),axis=0)

Loan_ID                        0
Loan_Amount_Requested          0
Loan_Grade                 25053
Length_Employed             7371
Home_Owner                 25349
Annual_Income              25102
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    88379
Number_Open_Accounts           0
Total_Accounts                 0
Area_Type                      0
Gender                         0
Interest_Rate                  0
dtype: int64

In [312]:
df_test.apply(lambda x: sum(x.isnull()),axis=0)

Loan_ID                        0
Loan_Amount_Requested          0
Loan_Grade                 16792
Length_Employed             4936
Home_Owner                 16711
Annual_Income              16898
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    58859
Number_Open_Accounts           0
Total_Accounts                 0
Area_Type                      0
Gender                         0
dtype: int64

In [313]:
df['Loan_Grade'].value_counts()

B    38540
C    35098
A    26887
D    24045
E    14686
Name: Loan_Grade, dtype: int64

In [314]:
df_test['Loan_Grade'].value_counts()

B    25775
C    23400
A    17849
D    15897
E     9828
Name: Loan_Grade, dtype: int64

In [315]:
df['Length_Employed'].value_counts()

10+ years    52915
2 years      14620
3 years      12974
< 1 year     12805
5 years      10814
1 year       10422
4 years       9784
7 years       9308
6 years       9094
8 years       7917
9 years       6285
Name: Length_Employed, dtype: int64

In [316]:
df_test['Length_Employed'].value_counts()

10+ years    35413
2 years       9800
3 years       8504
< 1 year      8328
5 years       7228
1 year        6914
4 years       6609
6 years       6257
7 years       6136
8 years       5198
9 years       4218
Name: Length_Employed, dtype: int64

In [317]:
df['Home_Owner'].value_counts()

Mortgage    70345
Rent        56031
Own         12525
Other          49
None           10
Name: Home_Owner, dtype: int64

In [318]:
df_test['Home_Owner'].value_counts()

Mortgage    46925
Rent        37511
Own          8354
Other          30
None           10
Name: Home_Owner, dtype: int64

In [319]:
df['Annual_Income'].value_counts()

60000.00     5462
50000.00     4783
65000.00     4012
40000.00     3856
70000.00     3667
45000.00     3614
75000.00     3434
80000.00     3429
55000.00     3320
90000.00     2637
100000.00    2618
85000.00     2377
35000.00     2350
30000.00     2248
120000.00    1981
42000.00     1905
52000.00     1861
48000.00     1786
110000.00    1581
72000.00     1552
95000.00     1411
36000.00     1215
150000.00    1191
62000.00     1173
38000.00     1165
54000.00     1069
32000.00     1055
58000.00     1047
125000.00    1013
56000.00     1007
             ... 
63624.00        1
111020.00       1
63599.00        1
63576.00        1
36628.80        1
63574.00        1
111045.00       1
63565.00        1
63552.00        1
111113.60       1
63655.00        1
63670.00        1
63819.00        1
63672.00        1
63814.40        1
110900.00       1
63790.00        1
110970.00       1
110999.00       1
63760.00        1
63755.00        1
63749.28        1
63748.00        1
63745.00        1
63732.00  

In [320]:
df_test['Annual_Income'].value_counts()

60000.00     3513
50000.00     3179
65000.00     2619
40000.00     2530
70000.00     2479
45000.00     2397
80000.00     2322
55000.00     2202
75000.00     2199
90000.00     1773
100000.00    1695
85000.00     1619
35000.00     1562
30000.00     1463
120000.00    1270
42000.00     1264
48000.00     1195
52000.00     1188
110000.00    1143
95000.00      984
72000.00      980
36000.00      897
38000.00      806
62000.00      795
150000.00     752
32000.00      741
68000.00      711
56000.00      709
54000.00      694
58000.00      671
             ... 
66516.00        1
66798.00        1
66540.72        1
66552.00        1
66577.00        1
66590.00        1
66607.00        1
66617.00        1
66622.00        1
66630.00        1
66640.00        1
66664.00        1
66665.64        1
66670.00        1
66681.00        1
66689.00        1
66690.00        1
66700.00        1
66720.00        1
66749.00        1
66755.00        1
66775.17        1
66779.00        1
66780.00        1
460000.00 

In [321]:
df['Months_Since_Deliquency'].value_counts()

9.0      1518
10.0     1510
14.0     1487
12.0     1484
8.0      1474
6.0      1440
13.0     1436
15.0     1433
7.0      1416
18.0     1390
16.0     1360
20.0     1333
11.0     1327
19.0     1325
17.0     1316
21.0     1297
27.0     1255
23.0     1224
26.0     1189
25.0     1189
22.0     1184
33.0     1156
31.0     1148
28.0     1147
29.0     1127
24.0     1125
30.0     1114
36.0     1104
5.0      1099
35.0     1083
         ... 
106.0       3
99.0        2
114.0       2
105.0       2
102.0       2
101.0       2
94.0        2
116.0       2
135.0       2
93.0        2
92.0        2
90.0        2
110.0       2
131.0       1
130.0       1
170.0       1
139.0       1
141.0       1
127.0       1
148.0       1
129.0       1
103.0       1
122.0       1
121.0       1
119.0       1
111.0       1
107.0       1
98.0        1
95.0        1
180.0       1
Name: Months_Since_Deliquency, dtype: int64

In [322]:
df_test['Months_Since_Deliquency'].value_counts()

9.0      1067
13.0     1060
8.0      1041
12.0      989
7.0       953
10.0      953
14.0      950
19.0      944
18.0      942
6.0       938
15.0      936
11.0      930
17.0      926
16.0      873
20.0      864
22.0      830
24.0      820
26.0      815
27.0      812
23.0      806
28.0      804
21.0      802
25.0      787
5.0       762
29.0      761
34.0      755
30.0      742
35.0      735
31.0      732
32.0      731
         ... 
88.0        4
96.0        4
95.0        4
90.0        4
92.0        4
86.0        4
91.0        3
101.0       2
87.0        2
85.0        2
89.0        2
126.0       1
130.0       1
131.0       1
143.0       1
133.0       1
135.0       1
140.0       1
122.0       1
124.0       1
97.0        1
118.0       1
115.0       1
109.0       1
106.0       1
103.0       1
99.0        1
94.0        1
93.0        1
149.0       1
Name: Months_Since_Deliquency, dtype: int64

In [323]:
df['Length_Employed'].fillna('10+ years ',inplace=True)
df['Home_Owner'].fillna('Mortgage',inplace=True)
df['Annual_Income'].fillna(df['Annual_Income'].mean(),inplace=True)
df['Months_Since_Deliquency'].fillna('0',inplace=True)


df_test['Length_Employed'].fillna('10+ years ',inplace=True)
df_test['Home_Owner'].fillna('Mortgage',inplace=True)
df_test['Annual_Income'].fillna(df['Annual_Income'].mean(),inplace=True)
df_test['Months_Since_Deliquency'].fillna('0',inplace=True)

In [324]:
df['Loan_Amount_Requested'] = df['Loan_Amount_Requested'].str.replace(',', '')
df['Loan_Amount_Requested'] = df['Loan_Amount_Requested'].astype(int)

df_test['Loan_Amount_Requested'] = df_test['Loan_Amount_Requested'].str.replace(',', '')
df_test['Loan_Amount_Requested'] = df_test['Loan_Amount_Requested'].astype(int)

In [325]:
df1=df.loc[df['Loan_Grade'].isnull(),:]
df2=df.dropna()
df2.describe()

,Loan_ID,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Number_Open_Accounts,Total_Accounts,Interest_Rate
count,1.392560e+05,139256.000000,1.392560e+05,139256.000000,139256.000000,139256.000000,139256.000000,139256.000000
mean,1.008210e+07,14339.856451,7.325795e+04,17.204112,0.781216,11.194527,25.065290,2.158772
std,4.746498e+04,8277.384684,5.562628e+04,7.842431,1.034458,4.996403,11.596461,0.738618
min,1.000000e+07,500.000000,4.000000e+03,0.000000,0.000000,1.000000,2.000000,1.000000
25%,1.004096e+07,8000.000000,4.882150e+04,11.370000,0.000000,8.000000,17.000000,2.000000
50%,1.008207e+07,12025.000000,7.000000e+04,16.840000,0.000000,10.000000,23.000000,2.000000
75%,1.012320e+07,20000.000000,8.200000e+04,22.770000,1.000000,14.000000,32.000000,3.000000
max,1.016431e+07,35000.000000,7.500000e+06,39.990000,8.000000,76.000000,156.000000,3.000000


In [339]:
df3=df_test.loc[df_test['Loan_Grade'].isnull(),:]
df4=df_test.dropna()
df4.describe()

,Loan_ID,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Number_Open_Accounts,Total_Accounts
count,9.274900e+04,92749.000000,9.274900e+04,92749.000000,92749.000000,92749.000000,92749.000000
mean,1.021908e+07,14366.417158,7.350843e+04,17.227747,0.790693,11.179463,25.078977
std,3.164382e+04,8315.119628,5.195235e+04,7.835789,1.042162,4.941177,11.603770
min,1.016431e+07,500.000000,3.000000e+03,0.000000,0.000000,0.000000,2.000000
25%,1.019164e+07,8000.000000,4.860000e+04,11.350000,0.000000,8.000000,17.000000
50%,1.021910e+07,12000.000000,7.000000e+04,16.860000,0.000000,10.000000,24.000000
75%,1.024646e+07,20000.000000,8.200000e+04,22.770000,1.000000,14.000000,32.000000
max,1.027385e+07,35000.000000,7.446395e+06,39.990000,8.000000,84.000000,124.000000


In [326]:
from sklearn import preprocessing

var_mod_train = ['Loan_Grade','Length_Employed','Home_Owner','Income_Verified','Purpose_Of_Loan','Area_Type','Gender']
var_mod_test= ['Length_Employed','Home_Owner','Income_Verified','Purpose_Of_Loan','Area_Type','Gender']
le = preprocessing.LabelEncoder()
lx=preprocessing.LabelEncoder()
for i in var_mod_train:
    df2[i] = le.fit_transform(df2[i].astype('str'))
for j in var_mod_test:
    df1[j] = lx.fit_transform(df1[j].astype('str'))

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [340]:

var_train = ['Loan_Grade','Length_Employed','Home_Owner','Income_Verified','Purpose_Of_Loan','Area_Type','Gender']
var_test= ['Length_Employed','Home_Owner','Income_Verified','Purpose_Of_Loan','Area_Type','Gender']
te = preprocessing.LabelEncoder()
tx=preprocessing.LabelEncoder()
for i in var_train:
    df4[i] = le.fit_transform(df4[i].astype('str'))
for j in var_test:
    df3[j] = lx.fit_transform(df3[j].astype('str'))

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [327]:
df2.apply(lambda x: sum(x.isnull()),axis=0)

Loan_ID                    0
Loan_Amount_Requested      0
Loan_Grade                 0
Length_Employed            0
Home_Owner                 0
Annual_Income              0
Income_Verified            0
Purpose_Of_Loan            0
Debt_To_Income             0
Inquiries_Last_6Mo         0
Months_Since_Deliquency    0
Number_Open_Accounts       0
Total_Accounts             0
Area_Type                  0
Gender                     0
Interest_Rate              0
dtype: int64

In [341]:
df4.apply(lambda x: sum(x.isnull()),axis=0)

Loan_ID                    0
Loan_Amount_Requested      0
Loan_Grade                 0
Length_Employed            0
Home_Owner                 0
Annual_Income              0
Income_Verified            0
Purpose_Of_Loan            0
Debt_To_Income             0
Inquiries_Last_6Mo         0
Months_Since_Deliquency    0
Number_Open_Accounts       0
Total_Accounts             0
Area_Type                  0
Gender                     0
dtype: int64

In [342]:
x_train= df2.drop(['Loan_Grade','Loan_ID','Interest_Rate'], axis=1)

y_train=df2['Loan_Grade']

x_test=df1.drop(['Loan_ID','Interest_Rate','Loan_Grade'],axis=1)

In [343]:
a_train= df4.drop(['Loan_Grade','Loan_ID'], axis=1)

b_train=df4['Loan_Grade']

a_test=df3.drop(['Loan_ID','Loan_Grade'],axis=1)

In [329]:
x_train.shape, y_train.shape, x_test.shape

((139256, 13), (139256,), (25053, 13))

In [344]:
a_train.shape, b_train.shape, a_test.shape

((92749, 13), (92749,), (16792, 13))

In [330]:
x_train.columns

Index(['Loan_Amount_Requested', 'Length_Employed', 'Home_Owner',
       'Annual_Income', 'Income_Verified', 'Purpose_Of_Loan', 'Debt_To_Income',
       'Inquiries_Last_6Mo', 'Months_Since_Deliquency', 'Number_Open_Accounts',
       'Total_Accounts', 'Area_Type', 'Gender'],
      dtype='object')

In [331]:
x_test.columns

Index(['Loan_Amount_Requested', 'Length_Employed', 'Home_Owner',
       'Annual_Income', 'Income_Verified', 'Purpose_Of_Loan', 'Debt_To_Income',
       'Inquiries_Last_6Mo', 'Months_Since_Deliquency', 'Number_Open_Accounts',
       'Total_Accounts', 'Area_Type', 'Gender'],
      dtype='object')

In [332]:
random_forest = RandomForestClassifier(n_estimators=100)

random_forest.fit(x_train, y_train)

pred = random_forest.predict(x_test)

random_forest.score(x_train, y_train)

1.0

In [347]:
random_forestx = RandomForestClassifier(n_estimators=100)

random_forestx.fit(a_train, b_train)

predx = random_forestx.predict(a_test)

random_forestx.score(a_train, b_train)

1.0

In [333]:
df1['Loan_Grade']=pred

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [348]:
df3['Loan_Grade']=predx

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [334]:
train=pd.read_csv("/resources/data/Interest Rate Prediction/train1.csv")
train.head()

,Loan_ID,Loan_Amount_Requested,Loan_Grade,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Area_Type,Gender,Interest_Rate
0,10000001,7000,0,11,4,68000.00000,2,0,18.37,0,0,9,14,1,0,1
1,10000002,30000,0,5,0,73331.15943,0,2,14.93,0,17,12,24,0,0,3
2,10000003,24725,3,8,0,75566.40000,1,2,15.88,0,0,12,16,1,1,3
3,10000004,16000,3,11,0,56160.00000,1,2,14.34,3,0,16,22,0,1,3
4,10000005,17000,0,9,3,96000.00000,1,2,22.17,1,0,19,30,1,0,1


In [353]:
test=pd.read_csv("/resources/data/Interest Rate Prediction/test1.csv")
test.head()

,Loan_ID,Loan_Amount_Requested,Loan_Grade,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Area_Type,Gender
0,10164310,27500,1,1,0,129000.0,0,2,12.87,0,68,10,37,1,1
1,10164311,26000,0,1,0,110000.0,2,1,11.37,0,0,6,23,1,1
2,10164312,6075,1,11,3,75000.0,0,2,6.83,2,0,5,20,0,1
3,10164313,12000,3,1,0,73000.0,1,2,7.76,0,0,6,8,2,1
4,10164314,35000,2,11,0,156000.0,2,2,9.62,0,26,9,21,2,1


In [354]:
fx_train= train.drop(['Loan_ID','Interest_Rate'], axis=1)

fy_train=train['Interest_Rate']

fx_test=test.drop(['Loan_ID'],axis=1)

In [355]:
fx_train.shape, fy_train.shape, fx_test.shape

((164309, 14), (164309,), (109541, 14))

In [357]:
forest = RandomForestClassifier(n_estimators=100)

forest.fit(fx_train, fy_train)

pre = forest.predict(fx_test)

forest.score(fx_train, fy_train)

1.0

In [ ]:
submission = pd.DataFrame({
        "Loan_ID": test["Loan_ID"],
        "Interest_Rate": pre
    })
submission.to_csv('/resources/data/Interest Rate Prediction/Sample_Submission.csv', index=False)